In [1]:
# Using cross entropy method to find an optimal policy for the Mountain Car Environment

import gym
import math
import numpy as np
from collections import deque
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [2]:
env = gym.make('MountainCarContinuous-v0')

print('Observation Space: ', env.observation_space)
print('Action Space: ', env.action_space)
print('Observation Space Lower limits: ', env.observation_space.low)
print('Observation Space Higher limits: ', env.observation_space.high)
print('Action Space Lower limits: ', env.action_space.low)
print('Action Space Higher limits: ', env.action_space.high)
print(env.observation_space.shape[0])
print(env.action_space.shape[0])

env.seed(1)
np.random.seed(1)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Observation Space:  Box(2,)
Action Space:  Box(1,)
Observation Space Lower limits:  [-1.2  -0.07]
Observation Space Higher limits:  [0.6  0.07]
Action Space Lower limits:  [-1.]
Action Space Higher limits:  [1.]
2
1


In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())

class Agent(nn.Module):
    def __init__(self, env, h_size=16):
        super(Agent, self).__init__()
        self.env = env
        self.s_size = env.observation_space.shape[0]
        self.h_size = h_size
        self.a_size = env.action_space.shape[0]

        #Layer definition
        self.fc1 = nn.Linear(self.s_size, self.h_size)
        self.fc2 = nn.Linear(self.h_size, self.a_size)
        
    def set_weights(self, weights):
        s_size = self.s_size
        h_size = self.h_size
        a_size = self.a_size
        # separate the weights for each layer
        fc1_end = (s_size*h_size)+h_size
        fc1_W = torch.from_numpy(weights[:s_size*h_size].reshape(s_size, h_size))
        fc1_b = torch.from_numpy(weights[s_size*h_size:fc1_end])
        fc2_W = torch.from_numpy(weights[fc1_end:fc1_end+(h_size*a_size)].reshape(h_size, a_size))
        fc2_b = torch.from_numpy(weights[fc1_end+(h_size*a_size):])

        # set the weights for each layer
        self.fc1.weight.data.copy_(fc1_W.view_as(self.fc1.weight.data))
        self.fc1.bias.data.copy_(fc1_b.view_as(self.fc1.bias.data))
        self.fc2.weight.data.copy_(fc2_W.view_as(self.fc2.weight.data))
        self.fc2.bias.data.copy_(fc2_b.view_as(self.fc2.bias.data))

    def get_weights_dim(self):
        return (self.s_size*self.h_size)+(self.h_size*1)+(self.a_size*self.h_size)+(self.a_size*1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        # We're using tanh as the final activation function here, 
        # since the action vector that this environment can take as input vary from -1 to +1. 
        # Look at the min and max of action space above.
        x = torch.tanh(self.fc2(x)) 
        return x.cpu().data

    def evaluate(self, weights, gamma = 1.0, max_t = 5000):
        self.set_weights(weights)
        episode_return = 0.0
        state = self.env.reset()
        for t in range(max_t):
            state = torch.from_numpy(state).float().to(device)
            action = self.forward(state)
            state, reward, done, _ = self.env.step(action)
            episode_return += reward * math.pow(gamma, t) # summing up the discounted rewards
            if done:
                break
        return episode_return
    
agent = Agent(env).to(device)
 
    

True


In [6]:
# Training the agent from scratch

def cem(n_iterations = 500, max_t = 1000, gamma = 1.0, print_every = 100, pop_size = 50, elite_frac = 0.2, sigma = 0.5):
    """PyTorch implementation of a cross-entropy method
    
    Params:
        n_iterations (int): maximum number of training iterations
        max_t (int): maximum number of timesteps per episode
        gamma (float): discount rate
        print_every (int): how often to print average score
        pop_size (int): size of population at each iteration
        elite_frac (float): percentage of top performers to use in update
        sigma (float): standard deviation of additive noise"""
    
    n_elite = int(pop_size*elite_frac)  # number of elite policies that gave good returns
    
    scores_deque = deque(maxlen = 100)
    scores = []
    best_weight = sigma*np.random.randn(agent.get_weights_dim())
    
    for i_iteration in range(1, n_iterations+1):
        weights_pop = [best_weight + (sigma*np.random.randn(agent.get_weights_dim())) for i in range(pop_size)]
        rewards = np.array([agent.evaluate(weights, gamma, max_t) for weights in weights_pop])
        
        elite_idxs = rewards .argsort()[-n_elite:]
        elite_weights = [weights_pop[i] for i in elite_idxs]
        best_weight = np.array(elite_weights).mean(axis = 0)
        
        reward = agent.evaluate(best_weight, gamma = 1.0)
        scores_deque.append(reward)
        scores.append(reward)
        
        torch.save(agent.state_dict(), 'checkpoint.pth')
        
        if i_iteration % print_every == 0:
            print('Episode {}\t Average Score: {:.2f}'.format(i_iteration, np.mean(scores_deque)))
        if np.mean(scores_deque)>=90.0:
            print('\nEnvironment solved in {:d} iterations! \t Average score: {:.2f}'.format(i_iteration-100, np.mean(scores_deque)))
            break
    return scores

scores = cem()
    

Episode 100	 Average Score: 79.96



Environment solved in 13 iterations! 	 Average score: 91.11
